In [ ]:
import functools
import json
import folium
import os
import csv
import math
import random
from pprint import pprint as pp

#### Generators

In [ ]:
def create_records_generator(start, end):
    while start < end:
        start += 1
        yield start


def create_records_array(start, end):
    res = []
    while start < end:
        start += 1
        res.append(start)
    return res

In [ ]:
generator = create_records_generator(3, 100000000)
for x in generator:
    if x % 10000000 == 0:
        print('from generator', x)

array = create_records_array(3, 100000000)
for n in array:
    if n % 10000000 == 0:
        print('from array', n)

If generators require less memory than lists, why not always use generators?

- lists should be used when you are iterating multiple times over the list (because they are cached).
- lists are needed when you need to access items out of order

#### Decorators

In [ ]:
import logging
          
def log_type_error(func):
    def wrapper(*args, **kwargs):
        try:
            order = func(*args, **kwargs)
            return order  
        except TypeError as e:
            logging.error("{0} error: {1}".format(func.__name__, e))
    return wrapper

@log_type_error
def concat_strings(*somewords):
    res = (' ').join(somewords)  
    return res            

res = concat_strings('polly\'s', 'pet', 'shop')
print(res)

res = concat_strings(7, 2)       
print(res)

In [ ]:
path = '/Users/cyrus/Documents/projects/codechix/code-chix-py-deck/customers.txt'
def parse_csv(path, delimiter='|'):
    with open(path, 'r') as f:
        lines = f.readlines()
        header = lines[0].strip().split(delimiter)
        records = [
            line.strip().split('|') for line in lines[1:]
            ]

    res = [
        dict(zip(header, record)) for record in records
        ]
    return res

records = parse_csv(path)
print(records[0:2])

In [ ]:
def make_geojson(func):
    @functools.wraps(func)
    def wrapper_convert_to_json(**kwargs):
        feature = {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "Point",
            "coordinates": [
                float(kwargs['customer']['easting']),
                float(kwargs['customer']['northing']),
            ]
          }
        }
        feature['properties'] = kwargs['customer']
        feature['properties']['distance'] = func(**kwargs)
        return feature
    return wrapper_convert_to_json

In [ ]:
@make_geojson
def calc_distance(store, customer):
    x1, x2 = float(store['easting']), float(customer['easting'])
    y1, y2 = float(store['northing']), float(customer['northing'])
    res = math.sqrt(
        ((x1-x2)**2) + ((y1-y2)**2)
    )
    return res

features = []
for rec in records:
    f = calc_distance(store={'easting': -122.5, 'northing': 37.51}, customer=rec)
    features.append(f)

In [ ]:
# def create_features(records):
#     features = []
#     for record in records:
#         feature = {
#           "type": "Feature",
#           "properties": {
#             "name": record["name"],
#             "pets": record["petcount"]
#           },
#           "geometry": {
#             "type": "Point",
#             "coordinates": [
#                 record["easting"],
#                 record["northing"]
#             ]
#           }
#         }
#         features.append(feature)
#     return features

In [ ]:
def map_results(features=[]):
    print(features[0])
    from pprint import pprint as pp
    featurejson = json.dumps(features)
    geo = {
          "type": "FeatureCollection",
          "features": features
        }
    
    gj = json.dumps(geo)

    easting = random.uniform(-122.112007, -122.056732)
    northing = random.uniform(37.367974, 37.424979)
    m = folium.Map(
        location=[northing, easting],
        tiles='OpenStreetMap',
        zoom_start=14
    )
    folium.GeoJson(
        gj,
        name='geojson'
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save('index2.html')
    
map_results(features)

In [ ]:
def main():
    path = os.path.join(os.getcwd(), 'customers.txt')
    customers = read_customer_data(path)
    features = create_features(customers)
    distance = calc_distance(start={'x': 5, 'y': 10}, end={'x': 2, 'y': 7})
    distance_no_dec = calc_distance_no_decorator(
        start={'x': 5, 'y': 10},
        end={'x': 2, 'y': 7})
    pp(distance)
    pp(distance_no_dec)
    map_results(features)

In [ ]:
if __name__ == '__main__':
    main()